<a href="https://colab.research.google.com/github/poongodimsa/DataScience-projects-for-vidiq/blob/main/Script_DS_Project1_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 1 -  Revenue by customer by month - V 1

### Version 2 - Specs


---



#### Let's call all files "Platform YY.MM.DD", so if we ran the Stripe output today, it would be called "Stripe 22.06.17", to be applied to PayPal and ChartMogul as well. Can you also add a folder in the "Results" folder called Archives, and put all old output files in there?
--> Will rename accordingly. I will code the script to move all the files under 'Results' to 'Archive' before creating a new file for the run. 

** Stripe Output: **


In the output file:Country: In the Stripe raw data files, column AK "card_country" provides a 2 digit country code. Pls use this.Plan Name: In the Stripe raw data files, col AL "statement_descriptor" provides the Plan. Pls use this for Customer Plan in the output.Plan Duration - Appears unavailable here
Customer Name - Appears unavailable hereLet's add an index column for Customer Email. This might work in lieu of a Name. Note we'll want to do this in PayPal too, so all Indexes match. That might mean Customer Unique ID will be something else for PayPal.


PayPal Output:

Country: 
There is a 2 digit country code like Stripe, we should try to have data conventions match if possible, so pls use this where we can. If this is N/A, your current output pulls from the PayPal raw data col "Country". However, there are many N/As. Finally, country is also provided in the last part of the "Shipping Address". Pls have your script try to pull from these three in this order, so if the 2 digit code is avail, pull that. If not, pull from "country". If that's not available, from col N "Shipping Address" by slicing the string off up to the last comma.  Between the 3 of these, we should be able to get nearly all countries. Plan: Can be found in Subject. It should either be Pro, Boost or Max. Your current output has some inaccurate info for plans - for example, we don't have a "Gold Plan", so not sure that that is. Duration: This can also be found in Subject, there is says "monthly" or "yearly". Just do a string search and if either of these are found in the row, add them to the duration.


### Version 1 - Specifications - For Paypal only

####  Final result:

1. Get EUR customer email addresses to USD amounts from EUR amounts

2. Remove /ignore all EUR amounts from the calculation. If you add all EUR (net) together. This will give you zero value

3. Ignore any amounts under "payments@vid.io" - Anway there will be no amounts for this email if you do the above two steps


#### Description:

Some of our customers pay in EUR. Since our default currency is USD, Paypal immediately converts these EUR payments to USD

The Problem we have; See the below screenshot taken from the 2022.02 Payroll data file. (A:5 to A:7)

Patrick is a EU customer. he paid us in EU currency. EUR 10.00
Paypal immediately converted to USD 9.84
You can also see that when you add both EUR amounts it will net set off ( Zero value)
So in these multi-currency cases, we need to get the USD values only. But the problem; there is no email address as the unique ID assigned to USD amounts. It is blank

The workaround here is 
In the above case, the Customer's email is knopey@gmx.de. But this is assigned to the EUR amount instead of the USD amount.

Transaction ID - 5RE84019JL6683131 This is the reference for the EUR amount. But we have a different reference for the USD amount. So we cannot use that get the customer email

Reference Txn ID - 5RE84019JL6683131 - Check the reference under "Reference Txn ID" for the USD amount now compare this with the above transaction ID. You will see there is a match
In Excel, we can use the "VlookUp" formula to get the customer email to USD amount using these two different IDs
Now please find a way to get the correct email ID to USD amount

**Important: Consider 'To Mail address' as unique id for debit transactions.**

### Version 0 - Specifications
---



Headers - Months in format “YYYY-MM”

Row Indexes - 
1.	Customer Name
2.	Platform (Stripe or PayPal)
3.	Customer Unique ID
4.	Customer Country (if available)
5.	Customer Plan (Pro or Boost; if available)
6.	Customer Plan Duration (Monthly or Yearly, if available)

Rows - Unique customers

Values - sum of total payments (received and refunded) per month for each customer

1.	Stripe - 

○	use "customer_id" as the unique ID to get the data
2.	PayPal

○	Important -----> Consider Only "Subscription Payment" & "Payment Refund" types ("E" column in the PayPal statements). Remove all the other types

○	Use "From Email Address" as the unique ID to get the data ( There is no special code for customer) 

3.	Ignore ChartMogul (“CM”) for now. This will be used in the second iteration.

Output

●	should be an excel file with one tab of revenue per customer per month

●	Sort by start month


Stripe

Customer Plan - You can map to "statement_descriptor" ( AL Column)

Customer Plan Duration - N/A - You Can keep this blank

Sum of total payments per customer ------> Always use Gross Amounts (G Column)

Paypal

Customer Unique ID - map to 'From Email Address? - "YES"

Customer Plan - You Can map the "Subject" /  "AD" Column 

Customer Plan Duration - N/A - You Can keep this blank

Amounts -----> Always use Gross Amounts / "H" Column



### Script
---

#### Imports,prepartions and functions

In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from datetime import date
from dateutil.relativedelta import *
import shutil


###### Preparations
---


In [ ]:
#Mount GDrive
from google.colab import drive
drive.mount('/content/gdrive')

#Set file paths
path = "/content/gdrive/My Drive/Data Science 2022/"

inpathST = path + "Stripe/"
inpathPP = path + "PayPal/"
inpathCM = path + "CM/"

outpath = path + "Results/"
archpath = path + "Results/Archives/"

#Create Results folder if missing
if not os.path.isdir(outpath):
  os.mkdir(outpath)

#Create archive folder if missing
if not os.path.isdir(archpath):
  os.mkdir(archpath)


Mounted at /content/gdrive


In [ ]:
#Move files if any in 'Results' to 'Archives'
for rfile in os.listdir(outpath):
  if os.path.isfile(outpath+rfile):
    src = outpath + rfile
    dest = archpath + rfile
    shutil.move(src, dest)

###### Functions common to all

------


In [ ]:
def get_month_headers(startmonth,monthcount):
  '''
  Function: Returns list of dates starting from 'startmonth' incrementing till 'monthcount' counts
  startdate: date in format 'YYYY-MM' as string datatype
  monthcount: Number of increments
  '''
  lstout = [] #Intialize
  start = datetime.strptime(startmonth,'%Y-%m') 
  for c in range(monthcount):
    lstout.append((start + relativedelta(months=+c)).strftime('%Y-%m'))
  return(lstout)

def gen_header():
  ''' Gnerates common header '''

  lstheader = [ "Customer Name","Platform","Customer Unique ID", "Customer Country", "Customer Plan", "Customer Plan Duration" ]
  lstmonths = get_month_headers('2020-01', 27)
  return(lstheader + lstmonths)

def getdate():
  ''' return todays date in yy.mm.dd format '''
  
  today = date.today()
  return today.strftime("%y.%m.%d")


Loop through the input files to calculate revenue per customer

#### Stripe

In [ ]:
c=0
for file in sorted(os.listdir(inpathST)):
  c+=1
  print(file)
  df = pd.read_csv(inpathST + file, na_filter=False, index_col=False)

  #Change the data type of gross to numeric
  df = df.astype({"gross": str})
  df['gross'] = df['gross'].str.replace(',','')
  df = df.astype({"gross": float})


  #*********** Calculate revenue per customer per month ******************
  #Apply Group by to input data
  sum_df = df.groupby(['customer_id' ], as_index=False).agg({'customer_name':'first', 'customer_id':'first',
                                                            'card_country':'first', 'statement_descriptor':'first','gross': 'sum'}) #grouped by dataframe'
  sum_df = sum_df.sort_values('customer_id')
  
  #Form revenue headers
  month = '20' + file[7:12].replace('.','-') #From input file name
  sum_df.rename(columns={'customer_name':'Customer Name', 'customer_id':'Customer Unique ID', 
                        'card_country': "Customer Country",'statement_descriptor':'Customer Plan', 'gross': month},inplace = True)
  
  
  
  #Create base dataframe from first file
  if c==1:
    final_df=sum_df.copy()
  else:
 
    #Merge using full outer join
    final_df = final_df.merge(sum_df, how='outer', on = 'Customer Unique ID')

    #************* V2 change: Keep the latest info for customer name, country and plan. If latest is null then copy from the previous record ***************
    #1. If gross is null for current month records means no data found in current file for that particular customer. Just ignore _y values. Copy _x to _y since we will drop _x records
    #2. If gross is not null but _y values are null, just copy _x values to _y
    #3. If gross is null in final_df then the customer is new. Do nothing. _x cols willbe deleted

    #Convert dataframe to dictionary and iterate which is faster than iterrows and itertuples
    final_dict = final_df.to_dict('records')

    # Check if _y values are null. If so copy _x value to _y. This way we get latest not null values
    for row in final_dict:
      if (row['Customer Name_y'] != row['Customer Name_y']) or (not  row['Customer Name_y']): #Checking for null in strings compare to itself returns false if nan
        row['Customer Name_y'] = row['Customer Name_x']
      if (row['Customer Country_y'] != row['Customer Country_y']) or (not row['Customer Country_y']):
        row['Customer Country_y'] = row['Customer Country_x']
      if (row['Customer Plan_y'] != row['Customer Plan_y']) or (not  row['Customer Plan_y']) : #Checking for null in strings
        row['Customer Plan_y'] = row['Customer Plan_x'] 

    #Convert dictinary back to dataframe
    final_df = pd.DataFrame.from_dict(final_dict)

    #Drop columns with '_x
    cols = [col for col in final_df.columns if col.lower()[-2:] != '_x']
    final_df=final_df[cols]

    #Remove '_y' from column name
    for col in final_df.columns:
      if col.lower()[-2:] == '_y':
        final_df.rename(columns = {col: col[:-2]}, inplace=True)

    
#insert 'Platform' and 'customer plan duration' cols
final_df.insert(1,"Platform", "Stripe")
final_df.insert(1,'Customer Plan Duration',"")


#Reorder the headers
lsthead = gen_header()
final_df = final_df.reindex(columns=lsthead)

#Sort by first month
final_df = final_df.sort_values('2020-01')

#Let's call all files "Platform YY.MM.DD"
dt = getdate()
outfilename = "Stripe " + dt + '.csv'

#Export data to excel without index
final_df.to_csv(outpath + outfilename, index = False)

print('done')

stripe_20.01.csv
stripe_20.02.csv
stripe_20.03.csv
stripe_20.04.csv
stripe_20.05.csv
stripe_20.06.csv
stripe_20.07.csv
stripe_20.08.csv
stripe_20.09.csv
stripe_20.10.csv


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


stripe_20.11.csv
stripe_20.12.csv
stripe_21.01.csv
stripe_21.02.csv
stripe_21.03.csv
stripe_21.04.csv
stripe_21.05.csv
stripe_21.06.csv
stripe_21.07.csv
stripe_21.08.csv
stripe_21.09.csv
stripe_21.10.csv
stripe_21.11.csv
stripe_21.12.csv
stripe_22.01.csv
stripe_22.02.csv
stripe_22.03.csv
done


#### PayPal

In [ ]:

c=0
for file in sorted(os.listdir(inpathPP)):
  c+=1
  print(file)
  #Read file to dataframe
  df = pd.read_csv(inpathPP + file, encoding = "ISO-8859-1", na_filter=False, index_col=False)

  #Consider Only "Subscription Payment" & "Payment Refund" types ("E" column in the PayPal statements). Remove all the other types
  #Include "General Currency Conversion" in the Filter to get USD for other currencies
  df = df[(df["Type"] == "Subscription Payment") | (df["Type"] == "Payment Refund") | (df["Type"] == "General Currency Conversion")].copy()

  df = df[["From Email Address","To Email Address","Name","Country Code", "Country","Shipping Address","Subject", "Gross","Balance Impact",
          "Currency","Type","Transaction ID","Reference Txn ID"]].copy()

  #Unique Id - If 'Balance Impact" is 'Credit' use 'From email address' else use 'To Email Address'
  #Convert dataframe to dictionary for iterating efficiency
  nonedata = [None] * len(df)
  dftemp = df.assign(Email = nonedata)
  
  tempdict = dftemp.to_dict('records')
  
  for row in tempdict:
    if row['Balance Impact'] == "Credit":
      #dftemp.loc[len(dftemp.index)] = [row["From Email Address"]]
      row['Email'] = row["From Email Address"]
    elif row['Balance Impact'] == "Debit":
      row['Email'] = row["To Email Address"]

    #V2- Keep country code, if it is empty, keep country, if empty strip from shipping address
    if (row['Country Code'] != row['Country Code']) or (not row['Country Code']):
      if (row['Country'] != row['Country']) or (not row['Country']):
        if (row['Shipping Address'] != row['Shipping Address']) or (not row['Shipping Address']):
          row['Country'] = ''
        else:
          #Get country from last part of shipping address
          row['Country'] = (row["Shipping Address"].split(',')[-1]).strip()
      else:
        row['Country'] = row['Country']
    else:
      row['Country'] = row['Country Code']

  
  df = pd.DataFrame.from_dict(tempdict)
  #Reset index to avoid null values
  df.reset_index(drop=True, inplace = True)

  #************************************* Logic to get email from other currency record to USD ******************
  #DF to copy email from 
  dffrom = df[ (df['Currency'] != "USD") & (( df['Type'] == "Subscription Payment") | (df["Type"] == "Payment Refund")) ].copy()
  # df copy to
  dfto = df[ (df['Currency'] == "USD") & ( df['Type'] == "General Currency Conversion")].copy()

  if not dfto.empty:
    dictfrom = dffrom.to_dict('records')
    dictto = dfto.to_dict('records')
    
    for rowf in dictfrom:
      txnid = rowf['Transaction ID']
      for rowt in dictto:
        if (rowt['Reference Txn ID'] == txnid):
          rowt['Email'] = rowf['Email']
          rowt['Name'] = rowf['Name']
          rowt['Country'] = rowf['Country']
          rowt['Subject'] = rowf['Subject']

    dfto = pd.DataFrame.from_dict(dictto)
    
    #Delete all empty 'Email' entries from dfto as it does not comply with 'Type' -'subs pay' or 'pay refund'
    dfto = dfto [(pd.isnull(dfto['Email']) == False)].copy()
    #Reset index to avoid null values
    dfto.reset_index(drop=True, inplace = True)
  
  df.reset_index(drop=True, inplace = True)

  dfnew = pd.concat([df,dfto], axis = 0)
  dfnew.reset_index(drop=True, inplace = True)

  
  #Now we have email ids for USD records
  #Delete all currencies other than 'USD' and all entries with blank 'Email'
  df = dfnew[(dfnew['Currency'] == "USD") & (pd.isnull(dfnew['Email']) == False)].copy()
  
  df = df[["Email","Name","Country","Subject", "Gross"]].copy()

  
  #Change the data type of gross to numeric
  df = df.astype({"Gross": str})
  df['Gross'] = df['Gross'].str.replace(',','')
  df = df.astype({"Gross": float})

  #************************* Calculate revenue per customer per month ******************
  #Apply Group by to input data
  sum_df = df.groupby(['Email' ], as_index=False).agg({'Name':'first', 'Email':'first',
                                                            'Country':'first', 'Subject':'first', 'Gross': 'sum'})     #grouped by dataframe'
  sum_df = sum_df.sort_values('Email')
  
  #Form revenue headers
  month =  file[7:14].replace('.','-')  #From input file name

  sum_df.rename(columns={'Name':'Customer Name', 'Email':'Customer Unique ID', 
                        'Country': "Customer Country",'Subject':'Customer Plan','Gross': month},inplace = True)
  
  
  #Create base dataframe from first file
  if c==1:
    final_df=sum_df.copy()
    
  else:
    
    #Merge using full outer join
    final_df = final_df.merge(sum_df, how='outer', on = 'Customer Unique ID')

    #************* V2 change: Keep the latest info for customer name, country and plan. If latest is null then copy from the previous record ***************
    #1. If gross is null for current month records means no data found in current file for that particular customer. Just ignore _y values. Copy _x to _y since we will drop _x records
    #2. If gross is not null but _y values are null, just copy _x values to _y
    #3. If gross is null in final_df then the customer is new. Do nothing. _x cols willbe deleted

    #Convert dataframe to dictionary and iterate which is faster than iterrows and itertuples
    final_dict = final_df.to_dict('records')

    # Check if _y values are null. If so copy _x value to _y. This way we get latest not null values
    for row in final_dict:
      if (row['Customer Name_y'] != row['Customer Name_y']) or (not  row['Customer Name_y']): #Checking for null in strings compare to itself returns false if nan
        row['Customer Name_y'] = row['Customer Name_x']
      if (row['Customer Country_y'] != row['Customer Country_y']) or (not row['Customer Country_y']):
        row['Customer Country_y'] = row['Customer Country_x']
      if (row['Customer Plan_y'] != row['Customer Plan_y']) or (not  row['Customer Plan_y']) : #Checking for null in strings
        row['Customer Plan_y'] = row['Customer Plan_x'] 
      
      
    #Convert dictinary back to dataframe
    final_df = pd.DataFrame.from_dict(final_dict)

    #Drop columns with '_x
    cols = [col for col in final_df.columns if col.lower()[-2:] != '_x']
    final_df=final_df[cols]

    #Remove '_y' from column name
    for col in final_df.columns:
      if col.lower()[-2:] == '_y':
        final_df.rename(columns = {col: col[:-2]}, inplace=True)

#insert 'Platform' and 'customer plan duration' cols
final_df.insert(1,"Platform", "PayPal")
final_df.insert(1,'Customer Plan Duration',"")

final_dict1 = final_df.to_dict('records')

# Check if _y values are null. If so copy _x value to _y. This way we get latest not null values
for row in final_dict1:
  #V2 - Extract plan duration from customer plan
  plan = str(row['Customer Plan'])
  if ((plan.lower().find('monthly')) != -1) or ((plan.lower().find('month')) != -1):
    row['Customer Plan Duration'] = 'Monthly'
  elif (plan.lower().find('yearly')) != -1:
    row['Customer Plan Duration'] = 'Yearly'

#Convert dictinary back to dataframe
final_df = pd.DataFrame.from_dict(final_dict1)

#Reorder the headers
lsthead = gen_header()
final_df = final_df.reindex(columns=lsthead)

#Sort by first month
final_df = final_df.sort_values('2020-01')

dt = getdate()
outfilename = "PayPal " + dt + '.csv'

#Export data to excel without index
final_df.to_csv(outpath + outfilename,index = False)

print('done')  

Paypal_2020.01.csv
Paypal_2020.02.csv
Paypal_2020.03.csv
Paypal_2020.04.csv
Paypal_2020.05.csv
Paypal_2020.06.csv
Paypal_2020.07.csv
Paypal_2020.08.csv
Paypal_2020.09.csv
Paypal_2020.10.csv
Paypal_2020.11.csv


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Paypal_2020.12.csv
Paypal_2021.01.csv
Paypal_2021.02.csv
Paypal_2021.03.csv
Paypal_2021.04.CSV
Paypal_2021.05.CSV
Paypal_2021.06.csv
Paypal_2021.07.csv
Paypal_2021.08.csv
Paypal_2021.09.CSV
Paypal_2021.10.csv
Paypal_2021.11.csv
Paypal_2021.12.csv
Paypal_2022.01.csv
Paypal_2022.02.csv
Paypal_2022.03.csv
done


#### ChartMogul

In [ ]:
c=0
for file in sorted(os.listdir(inpathCM)):
  c+=1
  print(file)
  df = pd.read_csv(inpathCM + file, na_filter=False, index_col=False)

  #Change the data type to numeric
  df = df.astype({"Line Item Value Account Currency": str})
  df['Line Item Value Account Currency'] = df["Line Item Value Account Currency"].str.replace(',','')
  df = df.astype({"Line Item Value Account Currency": float})
  

  #*********** Calculate revenue per customer per month ******************
  #Apply Group by to input data
  sum_df = df.groupby(['Customer External ID' ], as_index=False).agg({'Customer Name':'first', 'Customer External ID':'first',
                                                             'Line Item Name':'first','Line Item Value Account Currency': 'sum'}) #grouped by dataframe'
  sum_df = sum_df.sort_values('Customer External ID')
  
  #Form revenue headers
  month = '20' + file[3:8].replace('.','-') #From input file name
  sum_df.rename(columns={'Customer External ID':'Customer Unique ID', 
                        'Line Item Name':'Customer Plan', 'Line Item Value Account Currency': month},inplace = True)
  
  #Create base dataframe from first file
  if c==1:
    final_df=sum_df.copy()
  else:
 
    #Merge using full outer join
    final_df = final_df.merge(sum_df, how='outer', on = 'Customer Unique ID')

    #************* V2 change: Keep the latest info for customer name, country and plan. If latest is null then copy from the previous record ***************
    #1. If gross is null for current month records means no data found in current file for that particular customer. Just ignore _y values. Copy _x to _y since we will drop _x records
    #2. If gross is not null but _y values are null, just copy _x values to _y
    #3. If gross is null in final_df then the customer is new. Do nothing. _x cols willbe deleted

    #Convert dataframe to dictionary and iterate which is faster than iterrows and itertuples
    final_dict = final_df.to_dict('records')

    # Check if _y values are null. If so copy _x value to _y. This way we get latest not null values
    for row in final_dict:
      if (row['Customer Name_y'] != row['Customer Name_y']) or (not  row['Customer Name_y']): #Checking for null in strings compare to itself returns false if nan
        row['Customer Name_y'] = row['Customer Name_x']
      if (row['Customer Plan_y'] != row['Customer Plan_y']) or (not  row['Customer Plan_y']) : #Checking for null in strings
        row['Customer Plan_y'] = row['Customer Plan_x'] 

    #Convert dictinary back to dataframe
    final_df = pd.DataFrame.from_dict(final_dict)

    #Drop columns with '_x
    cols = [col for col in final_df.columns if col.lower()[-2:] != '_x']
    final_df=final_df[cols]

    #Remove '_y' from column name
    for col in final_df.columns:
      if col.lower()[-2:] == '_y':
        final_df.rename(columns = {col: col[:-2]}, inplace=True)

    
#insert 'Platform' and 'customer plan duration' cols
final_df.insert(1,"Platform", "ChartMogul")
final_df.insert(1,'Customer Country',"")

final_df.insert(1,'Customer Plan Duration',"")
final_dict1 = final_df.to_dict('records')
# Check if _y values are null. If so copy _x value to _y. This way we get latest not null values
for row in final_dict1:
  #V2 - Extract plan duration from customer plan
  plan = str(row['Customer Plan'])
  if ((plan.lower().find('monthly')) != -1) or ((plan.lower().find('month')) != -1):
    row['Customer Plan Duration'] = 'Monthly'
  elif ((plan.lower().find('yearly')) != -1) or ((plan.lower().find('year')) != -1):
    row['Customer Plan Duration'] = 'Yearly'

#Convert dictinary back to dataframe
final_df = pd.DataFrame.from_dict(final_dict1)

#Reorder the headers
lsthead = gen_header()
final_df = final_df.reindex(columns=lsthead)

#Sort by first month
final_df = final_df.sort_values('2020-01')

#Let's call all files "Platform YY.MM.DD"
dt = getdate()
outfilename = "CM " + dt + '.csv'

#To escape unicode error
#final_df = final_df.applymap(lambda x: x.encode('unicode_escape').
 #                decode('utf-8') if isinstance(x, str) else x)

#Export data to excel without index
final_df.to_csv(outpath + outfilename, index = False)

print('done')

CM_20.01.csv
CM_20.02.csv


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


CM_20.03.csv
CM_20.04.csv
CM_20.05.csv
CM_20.06.csv
CM_20.07.csv
CM_20.08.csv
CM_20.09.csv
CM_20.10.csv
CM_20.11.csv
CM_20.12.csv
CM_21.01.csv
CM_21.02.csv
CM_21.03.csv
CM_21.04.csv
CM_21.05.csv
CM_21.06.csv
CM_21.07.csv
CM_21.08.csv
CM_21.09.csv
CM_21.10.csv
CM_21.11.csv
CM_21.12.csv
CM_22.01.csv
CM_22.02.csv
CM_22.03.csv
done


#### Verify Stripe with CM

In [ ]:
#Load current Stripe and CM aggregated fiels
dt = getdate()

#Load Stripe and chartmogul files
infileS = "Stripe " + dt + '.csv'
infileCM = "CM " + dt + '.csv'
dfS = pd.read_csv(outpath + infileS)
dfCM = pd.read_csv(outpath + infileCM)

if not (dfS.empty and dfCM.empty):
  #Inner join on 'cutomer unique id'
  dfmerged = pd.merge(dfS,dfCM, on = "Customer Unique ID")
  #Remove unnecessary columns
  dfmerged.drop(['Platform_x', 'Customer Plan Duration_x','Platform_y','Customer Country_y'], axis = 1,inplace = True)
  #Rename '_x' columns with 'Stripe' and _y columns with 'CM'
  for col in dfmerged.columns:
    if col.lower()[-2:] == '_x':
      dfmerged.rename(columns = {col: (col[:-1] + 'Stripe')}, inplace=True)
    elif col.lower()[-2:] == '_y':
      dfmerged.rename(columns = {col: (col[:-1] + 'CM')}, inplace=True)
  
  #Reorder the headers so that stripe and CM data appear next to one another
  lstcustomer = ['Customer Unique ID', 'Customer Name_Stripe', 'Customer Name_CM', 'Customer Country_Stripe',
       'Customer Plan_Stripe', 'Customer Plan_CM','Customer Plan Duration_CM']
  lstmonths = get_month_headers('2020-01', 27)
  lstorderedmonths = []
  for mon in lstmonths:
    monS = str(mon) +'_Stripe'
    monCM = str(mon) +'_CM'
    lstorderedmonths.append(monS)
    lstorderedmonths.append(monCM)
  #List with headers in desired order
  lstfinal  = lstcustomer + lstorderedmonths 
  
  #THis dataframe has CM and Stripe info alternatively
  dfmerged = dfmerged.reindex(columns=lstfinal)

#*************** Find mismatches and write it seperately in a worksheet
  merge_dict = dfmerged.to_dict('records')
 
  dftemp = pd.DataFrame(columns = dfmerged.columns)
  p=0
  for row in merge_dict:
    
    for mon in lstmonths:
      monS = str(mon) +'_Stripe'
      monCM = str(mon) +'_CM'

      if (row[monS] != row[monCM]):
        if not((pd.isna(row[monS])) and (pd.isna(row[monCM]))) :
          if  not( (row[monS] == 0.0) and pd.isna(row[monCM])):
            if  not( (row[monCM] == 0.0) and pd.isna(row[monS])):
              Sdata = "%.2f" % float(row[monS])
              CMdata ="%.2f" % float(row[monCM])
              if Sdata != CMdata:
                df1 = {"Customer Unique ID":row["Customer Unique ID"],'Customer Name_Stripe':row['Customer Name_Stripe'], 'Customer Name_CM':row['Customer Name_CM'],
                      'Customer Country_Stripe':row['Customer Country_Stripe'],'Customer Plan_Stripe' : row['Customer Plan_Stripe'], 'Customer Plan_CM':row['Customer Plan_CM'],
                      'Customer Plan Duration_CM':row['Customer Plan Duration_CM'],  monS:row[monS], monCM : row[monCM]}
                dftemp = dftemp.append(df1, ignore_index = True)
                
    p += 1

  #dftemp.to_excel(outpath + 'Stripe Vs CM Differences.xlsx', index=False)
  #Sort by first month
  dfmerged = dfmerged.sort_values('2020-01_CM')
  dfmerged = dfmerged.applymap(lambda x: x.encode('unicode_escape').
                 decode('utf-8') if isinstance(x, str) else x)

  #dfmerged.to_csv(outpath + 'CM Vs Stripe.csv',index = False)
  #dfmerged.to_excel(outpath + 'Stripe Vs CM.xlsx',index = False)

  writer = pd.ExcelWriter(outpath + 'Stripe Vs CM.xlsx', engine = 'xlsxwriter')
  dfmerged.to_excel(writer, sheet_name = 'Stripe Vs CM', ignore_index = True)
  dftemp.to_excel(writer, sheet_name = 'Differences', ignore_index = True)
  writer.save()
  writer.close()

  print('done')

else:

  print("No input files found!")


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


done


/usr/local/lib/python3.7/dist-packages/xlsxwriter/workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [ ]:
writer = pd.ExcelWriter(outpath + 'Stripe Vs CM.xlsx', engine = 'xlsxwriter')
dfmerged.to_excel(writer, sheet_name = 'Stripe Vs CM', index = False)
dftemp.to_excel(writer, sheet_name = 'Differences', index = False)
writer.save()
writer.close()


/usr/local/lib/python3.7/dist-packages/xlsxwriter/workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


#### Verify Paypal with CM

In [ ]:
#Load Paypal and CM revenues by month
dt = getdate()

infilePP = "PayPal " + dt + '.csv'
infileCM = "CM " + dt + '.csv'
dfPP = pd.read_csv(outpath + infilePP)
dfCM = pd.read_csv(outpath + infileCM)

if not (dfPP.empty and dfCM.empty):
  #Inner join on 'cutomer unique id'
  dfmerged = pd.merge(dfPP,dfCM, on = "Customer Unique ID")
  #Remove unnecessary columns
  dfmerged.drop(['Platform_x', 'Customer Plan Duration_x','Platform_y','Customer Country_y'], axis = 1,inplace = True)
  #Rename '_x' columns with 'Stripe' and _y columns with 'CM'
  for col in dfmerged.columns:
    if col.lower()[-2:] == '_x':
      dfmerged.rename(columns = {col: (col[:-1] + 'Stripe')}, inplace=True)
    elif col.lower()[-2:] == '_y':
      dfmerged.rename(columns = {col: (col[:-1] + 'CM')}, inplace=True)
  
  #Reorder the headers so that stripe and CM data appear next to one another
  lstcustomer = ['Customer Unique ID', 'Customer Name_Stripe', 'Customer Name_CM', 'Customer Country_Stripe',
       'Customer Plan_Stripe', 'Customer Plan_CM','Customer Plan Duration_CM']
  lstmonths = get_month_headers('2020-01', 27)
  lstorderedmonths = []
  for mon in lstmonths:
    monS = str(mon) +'_Stripe'
    monCM = str(mon) +'_CM'
    lstorderedmonths.append(monS)
    lstorderedmonths.append(monCM)
    
  lstfinal  = lstcustomer + lstorderedmonths 
  
  dfmerged = dfmerged.reindex(columns=lstfinal)


  merge_dict = dfmerged.to_dict('records')
 
  dftemp = pd.DataFrame(columns = dfmerged.columns)
  p=0
  for row in merge_dict:
    
    for mon in lstmonths:
      monS = str(mon) +'_Stripe'
      monCM = str(mon) +'_CM'

      if (row[monS] != row[monCM]):
        if not((pd.isna(row[monS])) and (pd.isna(row[monCM]))) :
          if  not( (row[monS] == 0.0) and pd.isna(row[monCM])):
            if  not( (row[monCM] == 0.0) and pd.isna(row[monS])):
              Sdata = "%.2f" % float(row[monS])
              CMdata ="%.2f" % float(row[monCM])
              if Sdata != CMdata:
                df1 = {"Customer Unique ID":row["Customer Unique ID"],'Customer Name_Stripe':row['Customer Name_Stripe'], 'Customer Name_CM':row['Customer Name_CM'],
                      'Customer Country_Stripe':row['Customer Country_Stripe'],'Customer Plan_Stripe' : row['Customer Plan_Stripe'], 'Customer Plan_CM':row['Customer Plan_CM'],
                      'Customer Plan Duration_CM':row['Customer Plan Duration_CM'],  monS:row[monS], monCM : row[monCM]}
                dftemp = dftemp.append(df1, ignore_index = True)
                
    p += 1

  #dftemp.to_excel(outpath + 'Stripe Vs CM Differences.xlsx', index=False)
  #Sort by first month
  dfmerged = dfmerged.sort_values('2020-01_CM')
  dfmerged = dfmerged.applymap(lambda x: x.encode('unicode_escape').
                 decode('utf-8') if isinstance(x, str) else x)

  #dfmerged.to_csv(outpath + 'CM Vs Stripe.csv',index = False)
  #dfmerged.to_excel(outpath + 'Stripe Vs CM.xlsx',index = False)

  writer = pd.ExcelWriter(outpath + 'Stripe Vs CM.xlsx', engine = 'xlsxwriter')
  dfmerged.to_excel(writer, sheet_name = 'Stripe Vs CM', ignore_index = True)
  dftemp.to_excel(writer, sheet_name = 'Differences', ignore_index = True)
  writer.save()
  writer.close()

  print('done')

else:

  print("No input files found!")


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


done


/usr/local/lib/python3.7/dist-packages/xlsxwriter/workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [ ]:
writer = pd.ExcelWriter(outpath + 'Stripe Vs CM.xlsx', engine = 'xlsxwriter')
dfmerged.to_excel(writer, sheet_name = 'Stripe Vs CM', index = False)
dftemp.to_excel(writer, sheet_name = 'Differences', index = False)
writer.save()
writer.close()


/usr/local/lib/python3.7/dist-packages/xlsxwriter/workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


Excel Formatting to highlight the differences

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(outpath + 'CM Vs Stripe.xlsx', engine='xlsxwriter', engine_kwargs={'options':{'strings_to_formulas': False}})
dfmerged.to_excel(writer, index=False, sheet_name='report')

# Get access to the workbook and sheet
workbook = writer.book
worksheet = writer.sheets['report']

# Reduce the zoom a little
worksheet.set_zoom(90)
#Set column width
worksheet.set_column('A:G', 20)

# Add a format. Light red background with bolded text
format1 = workbook.add_format({'bg_color': '#FFC7CE', 'bold': True})

start_row=1
start_col=7
end_row=len(dfmerged)-1
end_col=8

worksheet.conditional_format(start_row, start_col, end_row, end_col, 
                             {'type': 'formula',
                              'criteria': '= H1 == I1', 
                              'format': format1})

writer.save()
writer.close()

/usr/local/lib/python3.7/dist-packages/xlsxwriter/workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


Excel Formatting to highlight the differences

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(outpath + 'CM Vs Stripe.xlsx', engine='xlsxwriter', engine_kwargs={'options':{'strings_to_formulas': False}})
dfmerged.to_excel(writer, index=False, sheet_name='report')

# Get access to the workbook and sheet
workbook = writer.book
worksheet = writer.sheets['report']

# Reduce the zoom a little
worksheet.set_zoom(90)
#Set column width
worksheet.set_column('A:G', 20)

# Add a format. Light red background with bolded text
format1 = workbook.add_format({'bg_color': '#FFC7CE', 'bold': True})

start_row=1
start_col=7
end_row=len(dfmerged)-1
end_col=8

worksheet.conditional_format(start_row, start_col, end_row, end_col, 
                             {'type': 'formula',
                              'criteria': '= H1 == I1', 
                              'format': format1})

writer.save()
writer.close()

/usr/local/lib/python3.7/dist-packages/xlsxwriter/workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
